In [1]:
import polars as pl 
import pandas as pd 
import polars.selectors as cs
import plotly.express as px

# Leitura dos dados e preparação para a análise

In [2]:
exp_vinho = (pl.read_csv("../data/ExpVinho.csv", separator=";") # lê os dados
             .select(~cs.starts_with("19") &                    # exclui colunas correspondentes ao século XX
                     ~cs.matches(r"(\b(200[0-6]))") &                   
                     ~cs.matches(r"(\b(200[0-6])_duplicated_0\b)")) # exclui colunas de 2000 a 2006    
             )
exp_vinho.head()

Id,País,2007,2007_duplicated_0,2008,2008_duplicated_0,2009,2009_duplicated_0,2010,2010_duplicated_0,2011,2011_duplicated_0,2012,2012_duplicated_0,2013,2013_duplicated_0,2014,2014_duplicated_0,2015,2015_duplicated_0,2016,2016_duplicated_0,2017,2017_duplicated_0,2018,2018_duplicated_0,2019,2019_duplicated_0,2020,2020_duplicated_0,2021,2021_duplicated_0,2022,2022_duplicated_0
i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,"""Afeganistão""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,46,0,0
2,"""África do Sul""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,95,4,21,0,0,0,0
3,"""Alemanha, Repú…",119512,238052,265742,429970,225086,393482,27715,138666,36070,144150,8189,56342,61699,265978,213348,761653,10680,44780,14012,68109,15467,87702,10794,45382,3660,25467,6261,32605,2698,6741,7630,45367
4,"""Angola""",25931,49753,25721,71083,54786,84235,33557,189891,13889,69001,2833,8861,1573,9300,12182,23124,1908,17089,7359,35390,10170,61680,477,709,345,1065,0,0,0,0,4068,4761
5,"""Anguilla""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
import re

# Define your regular expression pattern
pattern = r'((\d{4}|\d{4}_duplicated_0))'

# Your original list of strings
strings = ["2001", "2001_duplicated_0", "2002_name3", "other_text"]

# Define a function to use in re.sub()
def replace(match):
    year = match.group(1)
    name = match.group(2)
    new_name = f"{year}_qtd"
    return new_name

# Use re.sub() to apply the replacement
new_strings = [re.sub(pattern, replace, string) for string in strings]

# Print the renamed strings
for new_string in new_strings:
    print(new_string)


2001_qtd
2001_qtd_duplicated_0
2002_qtd_name3
other_text


In [3]:
old_cols = exp_vinho.columns
exp_vinho = (exp_vinho
    
    .rename({col: col.replace("_duplicated_0", "_val") for col in old_cols}) # renomeia as colunas para acrescentar o sufixo _val(US$)
    #.rename({col: re.sub(pattern, replace, col) for col in old_cols})
    .rename({"País":"Country"})
    
)

exp_vinho.head()

Id,Country,2007,2007_val,2008,2008_val,2009,2009_val,2010,2010_val,2011,2011_val,2012,2012_val,2013,2013_val,2014,2014_val,2015,2015_val,2016,2016_val,2017,2017_val,2018,2018_val,2019,2019_val,2020,2020_val,2021,2021_val,2022,2022_val
i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,"""Afeganistão""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,46,0,0
2,"""África do Sul""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,95,4,21,0,0,0,0
3,"""Alemanha, Repú…",119512,238052,265742,429970,225086,393482,27715,138666,36070,144150,8189,56342,61699,265978,213348,761653,10680,44780,14012,68109,15467,87702,10794,45382,3660,25467,6261,32605,2698,6741,7630,45367
4,"""Angola""",25931,49753,25721,71083,54786,84235,33557,189891,13889,69001,2833,8861,1573,9300,12182,23124,1908,17089,7359,35390,10170,61680,477,709,345,1065,0,0,0,0,4068,4761
5,"""Anguilla""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [114]:
exp_vinho_qtd = exp_vinho.select(cs.all() - cs.contains("_val"))
exp_vinho_qtd.head()

Id,Country,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,"""Afeganistão""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0
2,"""África do Sul""",0,0,0,0,0,0,0,0,0,0,0,0,26,4,0,0
3,"""Alemanha, Repú…",119512,265742,225086,27715,36070,8189,61699,213348,10680,14012,15467,10794,3660,6261,2698,7630
4,"""Angola""",25931,25721,54786,33557,13889,2833,1573,12182,1908,7359,10170,477,345,0,0,4068
5,"""Anguilla""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [204]:
exp_vinho_val = (
                (exp_vinho
                 .select(cs.starts_with(["Id", "Country"]) | cs.contains("_val") )
                 ).rename({col: col.replace("_val", "") for col in exp_vinho.columns}) 
                )
exp_vinho_val.head()

Id,Country,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,"""Afeganistão""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,46,0
2,"""África do Sul""",0,0,0,0,0,0,0,0,0,0,0,0,95,21,0,0
3,"""Alemanha, Repú…",238052,429970,393482,138666,144150,56342,265978,761653,44780,68109,87702,45382,25467,32605,6741,45367
4,"""Angola""",49753,71083,84235,189891,69001,8861,9300,23124,17089,35390,61680,709,1065,0,0,4761
5,"""Anguilla""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Estatísticas gerais - agregado

- Número de países na amostra: 128


## Quantidade de vinho exportado (1 Kg = 1 L)

In [74]:
# converts polars dataframe to pandas series to use with plotly express
def polars_df_to_pandas_series(df):
    index = df.columns
    df = (df.transpose()).to_pandas()
    df.index = index
    df = df.drop(index="describe")
    #print(df)

    return df 

In [220]:
# Sum of quantity by country
qty_by_country = []

for name, data in exp_vinho_qtd.group_by("Country"): 
        dictionary = {name: (data
                        .select(cs.all() - cs.starts_with(["Id", "Country"]))
                        .transpose()
                        .sum()
                        .to_numpy())}
        key = list(dictionary.keys())[0]
        value = list(dictionary.values())[0][0][0]        
        qty_by_country.append({"Country":key, "total_qty": value})
        

qty_by_country = pl.DataFrame(qty_by_country)
qty_by_country.head()


Country,total_qty
str,i64
"""Bolívia""",209406
"""Grécia""",9480
"""Argentina""",51012
"""Omã""",194
"""Moçambique""",383


In [223]:
px.bar(qty_by_country, x="Country", y="total_qty")

In [196]:
k = list(qtd_by_country[0].keys())[0]
v = list(qtd_by_country[0].values())[0][0][0]

print(k,v)

Camarões 1749


In [ ]:
qtd_by_coutry[0].pivot(values="baz", index="foo", columns="bar", aggregate_function="sum")

In [7]:
exp_vinho_qtd_summ = exp_vinho_qtd.select(cs.all() - cs.starts_with(["Id", "Country"])).describe()
exp_vinho_qtd_summ

describe,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",28798.046875,55610.15625,69851.46875,20275.804688,28243.125,43135.101562,177694.101562,58381.210938,21634.0625,34646.125,55746.929688,58484.867188,46365.828125,47486.625,76968.296875,85510.015625
"""std""",112790.554073,260556.007784,512646.746246,72911.605058,108366.034983,205931.510206,1.3477e6,193594.835251,88585.993054,170258.797445,400539.90942,487459.51628,343522.079741,348702.778141,639549.029833,641474.619049
"""min""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""50%""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,28.0,48.0,212.0
"""75%""",2448.0,6834.0,2143.0,1015.0,4899.0,5622.0,4092.0,3887.0,0.0,1214.0,2248.0,9744.0,2832.0,4805.0,8153.0,7914.0
"""max""",810038.0,2.352768e6,5.73228e6,478630.0,1.030254e6,2.103968e6,1.4795694e7,1.373747e6,741370.0,1.655417e6,4.27465e6,5.494321e6,3.826587e6,3.869243e6,7.192362e6,7.156293e6


In [95]:
exp_vinho_qtd = polars_df_to_pandas_series(exp_vinho_qtd_summ[[1,7],1:])
exp_vinho_qtd.columns = ["mean", "max"]
fig = px.scatter(exp_vinho_qtd, log_y=True, trendline='ols')
fig.show()

In [106]:
results = px.get_trendline_results(fig)
print(results)

  variable                                     px_fit_results
0     mean  <statsmodels.regression.linear_model.Regressio...
1      max  <statsmodels.regression.linear_model.Regressio...


In [108]:
results.query("variable == 'max'").px_fit_results.iloc[0].summary()


/home/juno/.cache/pypoetry/virtualenvs/winedata-Vmd1qRfz-py3.11/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1806: UserWarning:

kurtosistest only valid for n>=20 ... continuing anyway, n=16



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     1.635
Date:                Sat, 21 Oct 2023   Prob (F-statistic):              0.222
Time:                        14:59:24   Log-Likelihood:                -263.18
No. Observations:                  16   AIC:                             530.4
Df Residuals:                      14   BIC:                             531.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -4.991e+08   3.93e+08     -1.269      0.225   -1.34e+09    3.45e+08
x1          2.497e+05   1.95e+05      1.279      0.222   -1.69e+05    6.69e+05
==============================================================================
Omnibus:                       23.871   Durbin-Watson:                   2.233
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               29.435
Skew:                           2.225   Prob(JB):                     4.06e-07
Kurtosis:                       7.934   Cond. No.                     8.80e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.8e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
# mean by year without zero


## Valor de vinho exportado (US$) - qual a data-base?

In [110]:


exp_vinho_val_summ = exp_vinho_val.select(cs.all() - cs.starts_with(["Id", "Country"])).describe()
exp_vinho_val_summ

describe,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0,128.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",28798.046875,55610.15625,69851.46875,20275.804688,28243.125,43135.101562,177694.101562,58381.210938,21634.0625,34646.125,55746.929688,58484.867188,46365.828125,47486.625,76968.296875,85510.015625
"""std""",112790.554073,260556.007784,512646.746246,72911.605058,108366.034983,205931.510206,1.3477e6,193594.835251,88585.993054,170258.797445,400539.90942,487459.51628,343522.079741,348702.778141,639549.029833,641474.619049
"""min""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""50%""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,28.0,48.0,212.0
"""75%""",2448.0,6834.0,2143.0,1015.0,4899.0,5622.0,4092.0,3887.0,0.0,1214.0,2248.0,9744.0,2832.0,4805.0,8153.0,7914.0
"""max""",810038.0,2.352768e6,5.73228e6,478630.0,1.030254e6,2.103968e6,1.4795694e7,1.373747e6,741370.0,1.655417e6,4.27465e6,5.494321e6,3.826587e6,3.869243e6,7.192362e6,7.156293e6


In [111]:
exp_vinho_val = polars_df_to_pandas_series(exp_vinho_qtd_summ[[1,7],1:])
exp_vinho_val.columns = ["mean", "max"]
fig = px.scatter(exp_vinho_val, log_y=True, trendline='ols')
fig.show()

In [125]:
df = pl.DataFrame(
    {
        "foo": ["one", "one", "two", "two", "one", "two"],
        "bar": ["y", "y", "y", "x", "x", "x"],
        "baz": [1, 2, 3, 4, 5, 6],
    }
)
df

foo,bar,baz
str,str,i64
"""one""","""y""",1
"""one""","""y""",2
"""two""","""y""",3
"""two""","""x""",4
"""one""","""x""",5
"""two""","""x""",6


In [126]:
df.pivot(values="baz", index="foo", columns="bar", aggregate_function="sum")

foo,y,x
str,i64,i64
"""one""",3,5
"""two""",3,10
